In [1]:
import pandas as pd
import datetime
import os
import gc
import glob
import numpy as np
folder="/home/jian/Projects/Saatva/Test_Control_Pair/"

# P1 - Sales & Transaction

In [2]:
# week1 to week12 of transactions and sales 
start_date=datetime.datetime(2018,2,19).date() # Monday
end_date=datetime.datetime(2018,5,13).date() # Sunday

In [3]:
sales_df=pd.read_excel("/home/jian/Projects/Saatva/Transaction_Data/Sales_Jian_2018_5.24.2018.xlsx",dtype=str,sheetname="Sales_By_Brand")
sales_df=sales_df[['Order_id','Date','Time','Zip_Code','Sales','Source','Brand']]
sales_df=sales_df[sales_df['Brand']=='Saatva']
sales_df=sales_df.reset_index()
del sales_df['index']
sales_df['Date']=sales_df['Date'].apply(lambda x: datetime.datetime.strptime(x[0:10],"%Y-%m-%d").date())
sales_df['Time']=sales_df['Time'].apply(lambda x: datetime.datetime.strptime(x,"%H:%M:%S").time())
sales_df['Sales']=sales_df['Sales'].astype(float)
sales_df.shape

(32153, 7)

In [4]:
zip_dma=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
zip_dma=zip_dma.iloc[:,[0,2]].rename(columns={"CODE":"Zip_Code","NAME":"DMA"})
zip_dma=zip_dma.drop_duplicates()

In [5]:
sales_df=pd.merge(sales_df,zip_dma,on="Zip_Code",how="left")

In [6]:
selected_25_DMA=pd.read_excel("/home/jian/Projects/Saatva/Otherinput/selected_25_DMAs.xlsx",dtype=str)
selected_25_DMA=selected_25_DMA[selected_25_DMA['In_TV_25_Index']=="In_Top25_TV"]

In [7]:
sales_25_DMA=sales_df[sales_df['DMA'].isin(selected_25_DMA['dma'])]
sales_25_DMA=sales_25_DMA[(sales_25_DMA['Date']>=start_date) & (sales_25_DMA['Date']<=end_date)].reset_index()
del sales_25_DMA['index']

In [8]:
sales_25_DMA_sales=sales_25_DMA.groupby("DMA")['Sales'].sum().to_frame().reset_index()
sales_25_DMA_transctions=sales_25_DMA.groupby("DMA")['Sales'].count().to_frame().reset_index().rename(columns={"Sales":"Transactions"})

sales_25_DMA_P1=pd.merge(sales_25_DMA_sales,sales_25_DMA_transctions,on="DMA",how="left")

# P2 - Adult 35-54

In [9]:
zip_pop=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset.csv",dtype=str)

zip_pop=zip_pop[['ZIP_CODE','Estimate; SEX AND AGE - 35 to 44 years','Estimate; SEX AND AGE - 45 to 54 years']]
zip_pop['Estimate; SEX AND AGE - 35 to 44 years']=zip_pop['Estimate; SEX AND AGE - 35 to 44 years'].astype(float).fillna(0)
zip_pop['Estimate; SEX AND AGE - 45 to 54 years']=zip_pop['Estimate; SEX AND AGE - 45 to 54 years'].astype(float).fillna(0)
zip_pop['Adult_35_to_54']=zip_pop['Estimate; SEX AND AGE - 35 to 44 years']+zip_pop['Estimate; SEX AND AGE - 45 to 54 years']
zip_pop=zip_pop[['ZIP_CODE','Adult_35_to_54']]
zip_pop['ZIP_CODE']=zip_pop['ZIP_CODE'].astype(str).apply(lambda x:x.zfill(5))
zip_pop=zip_pop.rename(columns={"ZIP_CODE":"Zip_Code"})

In [10]:
zip_dma_Adult=pd.merge(zip_dma,zip_pop,on="Zip_Code",how="left")
zip_dma_Adult['Adult_35_to_54']=zip_dma_Adult['Adult_35_to_54'].fillna(0)

In [11]:
Adult_25_DMA_P2=zip_dma_Adult.groupby("DMA")['Adult_35_to_54'].sum().to_frame().reset_index()

In [12]:
dma_cleaning=pd.read_excel("/home/jian/Docs/DMA cleaning.xlsx",dtype=str,sheetname="new clean")[['dma','cleaned dma']].drop_duplicates()
dma_cleaning_Nielsen=dma_cleaning[dma_cleaning['dma'].isin(zip_dma['DMA'])].drop_duplicates()

In [13]:
dma_cleaning_Nielsen.head(2)

,dma,cleaned dma
4,ABILENE-SWEETWATER,"Abilene-Sweetwater, TX"
16,ALBANY-SCHENECTADY-TROY,"Albany-Schenectady-Troy, NY"


In [14]:
dma_cleaning_Nielsen=dma_cleaning_Nielsen[['dma','cleaned dma']]

# P3 - GA:Sessions

In [15]:
import google_analytics_reporting_api as ga
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [16]:
# Saatva.com
# date DMA Session
ga_data_saatva=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=str(start_date), end_date=str(end_date),
                                            ga_metrics=['ga:sessions','ga:bounceRate','ga:pageviews','ga:bounces'],
                                            ga_dimensions=['ga:date','ga:metro'])


ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(15144, 8)

In [17]:
# Saatvamattress.com
# date DMA Session
ga_data_SM=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=str(start_date), end_date=str(end_date),
                                            ga_metrics=['ga:sessions','ga:bounceRate','ga:pageviews','ga:bounces'],
                                            ga_dimensions=['ga:date','ga:metro'])


ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(18792, 8)

In [18]:
ga_both=ga_data_saatva.append(ga_data_SM)
ga_both=ga_both.rename(columns={'ga:metro':"dma"})
ga_both=pd.merge(ga_both,dma_cleaning,on="dma",how="left")
del ga_both['dma']
ga_both=pd.merge(ga_both,dma_cleaning_Nielsen,on="cleaned dma",how="left")
del ga_both['cleaned dma']
ga_both=ga_both.rename(columns={"dma":"DMA"})

In [19]:
ga_both['ga:sessions']=ga_both['ga:sessions'].astype(int)
ga_both['ga:pageviews']=ga_both['ga:pageviews'].astype(int)
ga_both['ga:bounces']=ga_both['ga:bounces'].astype(int)

# 229 DMAs in total including "not set"
Session_DMA_P3=ga_both.groupby('DMA')['ga:sessions'].sum().to_frame().reset_index()
Session_25_DMA_P3=Session_DMA_P3[Session_DMA_P3['DMA'].isin(sales_25_DMA['DMA'])]

# P4 - BounceRate | PageViews

In [20]:
PV_BR_DMA_P4=ga_both.groupby('DMA')['ga:pageviews','ga:bounces','ga:sessions'].sum().reset_index()
PV_BR_DMA_P4['BournceRate']=PV_BR_DMA_P4['ga:bounces']/PV_BR_DMA_P4['ga:sessions']
PV_BR_DMA_P4=PV_BR_DMA_P4[['DMA','ga:pageviews','BournceRate']]
PV_BR_25_DMA_P4=PV_BR_DMA_P4[PV_BR_DMA_P4['DMA'].isin(sales_25_DMA['DMA'])]

# P5 - ARI

In [30]:
[x for x in  zip_dma['DMA'].unique().tolist() if x not in  sales_df['DMA'].unique().tolist()]

['HONOLULU', 'FAIRBANKS', 'JUNEAU']

In [32]:
sales_ARI=sales_df[(sales_df['Date']>=start_date) & (sales_df['Date']<=end_date)]
sales_ARI['DMA']=sales_ARI['DMA'].fillna("nan")
sales_ARI=sales_ARI[sales_ARI['DMA']!="nan"]

sales_ARI['Date_Delta']=sales_ARI['Date'].apply(lambda x: (x-start_date).days)
sales_ARI['Week_Num']=(np.floor(sales_ARI['Date_Delta']/7)+1).astype(int)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
sales_ARI_7_9_11=sales_ARI[sales_ARI['Week_Num'].isin([7,9,11])]
sales_ARI_8_10_12=sales_ARI[sales_ARI['Week_Num'].isin([8,10,12])]

array([datetime.date(2018, 5, 6), datetime.date(2018, 5, 5),
       datetime.date(2018, 5, 4), datetime.date(2018, 5, 3),
       datetime.date(2018, 5, 2), datetime.date(2018, 5, 1),
       datetime.date(2018, 4, 30), datetime.date(2018, 4, 22),
       datetime.date(2018, 4, 21), datetime.date(2018, 4, 20),
       datetime.date(2018, 4, 19), datetime.date(2018, 4, 18),
       datetime.date(2018, 4, 17), datetime.date(2018, 4, 16),
       datetime.date(2018, 4, 8), datetime.date(2018, 4, 7),
       datetime.date(2018, 4, 6), datetime.date(2018, 4, 5),
       datetime.date(2018, 4, 4), datetime.date(2018, 4, 3),
       datetime.date(2018, 4, 2)], dtype=object)

In [44]:
sales_ARI_all_DMA_7_9_11_sales=sales_ARI_7_9_11.groupby("DMA")['Sales'].sum().to_frame().reset_index()
sales_ARI_all_DMA_8_10_12_sales=sales_ARI_8_10_12.groupby("DMA")['Sales'].sum().to_frame().reset_index()
sales_ARI_all_DMA_7_9_11_sales=sales_ARI_all_DMA_7_9_11_sales.rename(columns={"Sales":"Base_Sales"})
sales_ARI_all_DMA_8_10_12_sales=sales_ARI_all_DMA_8_10_12_sales.rename(columns={"Sales":"Campaign_Sales"})


sales_ARI_all_DMA_7_9_11_trans=sales_ARI_7_9_11.groupby("DMA")['Sales'].count().to_frame().reset_index()
sales_ARI_all_DMA_8_10_12_trans=sales_ARI_8_10_12.groupby("DMA")['Sales'].count().to_frame().reset_index()
sales_ARI_all_DMA_7_9_11_trans=sales_ARI_all_DMA_7_9_11_trans.rename(columns={"Sales":"Base_Trans"})
sales_ARI_all_DMA_8_10_12_trans=sales_ARI_all_DMA_8_10_12_trans.rename(columns={"Sales":"Campaign_Trans"})

All_DMA_ARI_df=pd.merge(sales_ARI_all_DMA_7_9_11_sales,sales_ARI_all_DMA_8_10_12_sales,on="DMA",how="left")
All_DMA_ARI_df=pd.merge(All_DMA_ARI_df,sales_ARI_all_DMA_7_9_11_trans,on="DMA",how="left")
All_DMA_ARI_df=pd.merge(All_DMA_ARI_df,sales_ARI_all_DMA_8_10_12_trans,on="DMA",how="left")

In [52]:
All_DMA_ARI_df=All_DMA_ARI_df.fillna(0)
All_DMA_ARI_df['Sales_ARI']=np.nan
All_DMA_ARI_df['Trans_ARI']=np.nan
for i in range(len(All_DMA_ARI_df)):
    All_DMA_ARI_df['Sales_ARI'][i]=(All_DMA_ARI_df['Campaign_Sales'][i]/All_DMA_ARI_df['Base_Sales'][i])/(All_DMA_ARI_df['Campaign_Sales'].sum()/All_DMA_ARI_df['Base_Sales'].sum())*100
    All_DMA_ARI_df['Trans_ARI'][i]=(All_DMA_ARI_df['Campaign_Trans'][i]/All_DMA_ARI_df['Base_Trans'][i])/(All_DMA_ARI_df['Campaign_Trans'].sum()/All_DMA_ARI_df['Base_Trans'].sum())*100

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
ARI_25_DMA_P5=All_DMA_ARI_df[All_DMA_ARI_df['DMA'].isin(PV_BR_25_DMA_P4['DMA'])][['DMA',"Sales_ARI","Trans_ARI"]]

# Calculate Diff

In [180]:
data_5P=pd.merge(sales_25_DMA_P1,Adult_25_DMA_P2,on="DMA",how="left")
data_5P=pd.merge(data_5P,Session_25_DMA_P3,on="DMA",how="left")
data_5P=pd.merge(data_5P,PV_BR_25_DMA_P4,on="DMA",how="left")
data_5P=pd.merge(data_5P,ARI_25_DMA_P5,on="DMA",how="left")


In [181]:
data_5P_C=data_5P.copy()
data_5P_T=data_5P.copy()
data_5P_C.columns=["C_"+x for x in data_5P_C.columns.tolist()]
data_5P_T.columns=["T_"+x for x in data_5P_T.columns.tolist()]

In [182]:
df_score=pd.DataFrame()
for DMA in data_5P['DMA']:
    df=data_5P[['DMA',"Sales"]]
    df.columns=["C_DMA","T_DMA"]
    df['T_DMA']=DMA
    df_score=df_score.append(df)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [183]:
data_5P_score=pd.merge(df_score,data_5P_C,on="C_DMA",how="left")
data_5P_score=pd.merge(data_5P_score,data_5P_T,on="T_DMA",how="left")

In [184]:
# Paired Calculation 1 - Pageviews for P4

for i in range(8):
    col="diff_"+data_5P_score.columns.tolist()[2+i][2:]
    data_5P_score[col]=(data_5P_score[data_5P_score.columns.tolist()[i+10]]-data_5P_score[data_5P_score.columns.tolist()[i+2]])/data_5P_score[data_5P_score.columns.tolist()[i+2]]

data_5P_score["Score_with_PageView"]=0.1*abs(data_5P_score['diff_Sales'])+0.1*abs(data_5P_score['diff_Transactions'])+\
                                    0.2*abs(data_5P_score['diff_Adult_35_to_54'])+\
                                    0.2*abs(data_5P_score['diff_ga:sessions'])+\
                                    0.2*abs(data_5P_score['diff_ga:pageviews'])+\
                                    0.1*abs(data_5P_score['diff_Sales_ARI'])+0.1*abs(data_5P_score['diff_Trans_ARI'])

data_5P_score["Score_with_BounceRate"]=0.1*abs(data_5P_score['diff_Sales'])+0.1*abs(data_5P_score['diff_Transactions'])+\
                                    0.2*abs(data_5P_score['diff_Adult_35_to_54'])+\
                                    0.2*abs(data_5P_score['diff_ga:sessions'])+\
                                    0.2*abs(data_5P_score['diff_BournceRate'])+\
                                    0.1*abs(data_5P_score['diff_Sales_ARI'])+0.1*abs(data_5P_score['diff_Trans_ARI'])
    

In [185]:
data_5P_score_PageView_Rank=data_5P_score.sort_values("Score_with_PageView")
data_5P_score_BournceRate_Rank=data_5P_score.sort_values("Score_with_BounceRate")


In [186]:
writer=pd.ExcelWriter(folder+"Saatva_markets for location test_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [187]:
data_5P_score_PageView_Rank.to_excel(writer,"1_scored",index=False)
data_5P_score_BournceRate_Rank.to_excel(writer,"2_scored",index=False)
data_5P_score.to_excel(writer,"diff_scored",index=False)
data_5P.to_excel(writer,"data",index=False)

In [188]:
writer.save()